In [16]:
import rosbag2_py
from rclpy.serialization import serialize_message
from sensor_msgs.msg import PointCloud2, PointField
import std_msgs.msg as std_msgs
import numpy as np
from builtin_interfaces.msg import Time
import ros2_numpy as rnpy

In [2]:
bag_writer = rosbag2_py.SequentialWriter()
storage_options = rosbag2_py._storage.StorageOptions(
            uri='waymo_10504764403039842352_460_000_480_000',
            storage_id='sqlite3')
converter_options = rosbag2_py._storage.ConverterOptions('', '')
bag_writer.open(storage_options, converter_options)
pcd_topic_info = rosbag2_py._storage.TopicMetadata(
            name='/points2',
            type='sensor_msgs/msg/PointCloud2',
            serialization_format='cdr')
bag_writer.create_topic(pcd_topic_info)

RuntimeError: Database directory already exists (waymo_10504764403039842352_460_000_480_000), can't overwrite existing database

In [36]:
np.random.seed(9999)
cld_np = np.random.rand(4,10)
dt = {'names': ('x', 'y', 'z', 'i'), 'formats': (np.float32, np.float32, np.float32, np.float32)}
cld_np = np.rec.fromarrays(cld_np, dtype=dt)
cld_np

rec.array([(0.82338905, 0.91304946, 0.8527295 , 0.33330482),
           (0.21860275, 0.7711041 , 0.16602555, 0.58254296),
           (0.03146101, 0.28836122, 0.48247832, 0.6793682 ),
           (0.54592615, 0.41544133, 0.5510306 , 0.54465324),
           (0.15445562, 0.33149043, 0.60251755, 0.91268677),
           (0.18696329, 0.3663267 , 0.43566513, 0.7822505 ),
           (0.4223477 , 0.31219533, 0.4723211 , 0.7107666 ),
           (0.6055525 , 0.5804458 , 0.86600107, 0.09205964),
           (0.49235412, 0.11295173, 0.5376863 , 0.9522484 ),
           (0.6441191 , 0.32587487, 0.48458973, 0.22353162)],
          dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('i', '<f4')])

In [37]:
ts = Time(sec=100, nanosec=12000)
# pcd = point_cloud(cld_np, ts)
pcd = rnpy.array_to_pointcloud2(cld_np, ts, '/lidar')
pcd

sensor_msgs.msg.PointCloud2(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=100, nanosec=12000), frame_id='/lidar'), height=1, width=10, fields=[sensor_msgs.msg.PointField(name='x', offset=0, datatype=7, count=1), sensor_msgs.msg.PointField(name='y', offset=4, datatype=7, count=1), sensor_msgs.msg.PointField(name='z', offset=8, datatype=7, count=1), sensor_msgs.msg.PointField(name='i', offset=12, datatype=7, count=1)], is_bigendian=False, point_step=16, row_step=160, data=[160, 201, 82, 63, 156, 189, 105, 63, 123, 76, 90, 63, 238, 166, 170, 62, 102, 217, 95, 62, 20, 103, 69, 63, 154, 2, 42, 62, 137, 33, 21, 63, 66, 221, 0, 61, 21, 164, 147, 62, 102, 7, 247, 62, 19, 235, 45, 63, 209, 193, 11, 63, 186, 180, 212, 62, 87, 16, 13, 63, 101, 110, 11, 63, 157, 41, 30, 62, 29, 185, 169, 62, 151, 62, 26, 63, 215, 165, 105, 63, 78, 115, 63, 62, 44, 143, 187, 62, 128, 15, 223, 62, 146, 65, 72, 63, 245, 61, 216, 62, 17, 216, 159, 62, 18, 212, 241, 62, 205, 244, 53, 63, 125, 5, 27, 

In [38]:
rnpy.pointcloud2_to_array(pcd)

array([(0.82338905, 0.91304946, 0.8527295 , 0.33330482),
       (0.21860275, 0.7711041 , 0.16602555, 0.58254296),
       (0.03146101, 0.28836122, 0.48247832, 0.6793682 ),
       (0.54592615, 0.41544133, 0.5510306 , 0.54465324),
       (0.15445562, 0.33149043, 0.60251755, 0.91268677),
       (0.18696329, 0.3663267 , 0.43566513, 0.7822505 ),
       (0.4223477 , 0.31219533, 0.4723211 , 0.7107666 ),
       (0.6055525 , 0.5804458 , 0.86600107, 0.09205964),
       (0.49235412, 0.11295173, 0.5376863 , 0.9522484 ),
       (0.6441191 , 0.32587487, 0.48458973, 0.22353162)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('i', '<f4')])